Mixture of pandas and sqlite[https://www.sqlite.org/about.html] to examine the df 

limitations of sqlite: I can't do much of the data cleansing in sqlite. I'll also include what the sql statement would be to achieve the same results anytime i use pandas. 2 links

https://www.simplilearn.com/tutorials/sql-tutorial/what-is-sqlite#:~:text=Limitations%20of%20SQLite&text=SQLite%20only%20allows%20normal%20file,support%20FOR%20EACH%20STATEMENT%20triggers.

https://www.sqlite.org/limits.html


### Importing Neccessary Libraries

In [6]:
import pandas as pd
import sqlite3
import pandasql as ps #sql package

### Reading In the Dataset 

In [7]:
df = pd.read_csv('Daily_Inmates_In_Custody.csv')
df

,INMATEID,ADMITTED_DT,DISCHARGED_DT,CUSTODY_LEVEL,BRADH,RACE,GENDER,AGE,INMATE_STATUS_CODE,SEALED,SRG_FLG,TOP_CHARGE,INFRACTION
0,20197307,03/23/2023 02:49:00 PM,NaN,MAX,Y,B,M,21,DE,N,Y,110-125.25,Y
1,20223604,03/16/2023 09:14:00 PM,NaN,MED,Y,O,M,70,DE,N,N,130.50,N
2,20031187,06/11/2023 12:38:00 AM,NaN,MED,Y,O,M,31,DE,N,Y,220.16,N
3,20223353,03/02/2023 01:57:34 AM,NaN,NaN,Y,O,M,23,DE,N,N,110-125.25,Y
4,20070053,09/05/2021 12:50:57 AM,NaN,MED,N,O,M,32,DE,N,N,110-125.25,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6141,20140924,07/28/2022 02:08:38 AM,NaN,MED,Y,W,M,44,CS,N,N,270.25,N
6142,20008122,07/13/2023 08:47:04 PM,NaN,MIN,N,O,M,32,DE,N,N,120.00,N
6143,20206710,07/13/2023 07:21:01 PM,NaN,MAX,N,O,M,23,DE,N,N,120.05,N
6144,20220543,09/07/2022 04:43:55 PM,NaN,MAX,N,O,M,20,DE,N,N,NaN,Y


### Examining the Dataframes Info

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6146 entries, 0 to 6145
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   INMATEID            6146 non-null   int64  
 1   ADMITTED_DT         6146 non-null   object 
 2   DISCHARGED_DT       0 non-null      float64
 3   CUSTODY_LEVEL       6113 non-null   object 
 4   BRADH               6146 non-null   object 
 5   RACE                6128 non-null   object 
 6   GENDER              6128 non-null   object 
 7   AGE                 6146 non-null   int64  
 8   INMATE_STATUS_CODE  6146 non-null   object 
 9   SEALED              6146 non-null   object 
 10  SRG_FLG             6146 non-null   object 
 11  TOP_CHARGE          5975 non-null   object 
 12  INFRACTION          6146 non-null   object 
dtypes: float64(1), int64(2), object(10)
memory usage: 624.3+ KB


Takeaways:

- The discharged date column has 0 values and should be removed

- The admitted date column needs its data type to be converted to a datetime and then broken out

The SQL alternative for the above code depends on the specific SQL Database you are using. For example, in MySql, this format would query your table data types:

In [ ]:
SELECT DATA_TYPE from INFORMATION_SCHEMA.COLUMNS where
    table_schema = 'yourDatabaseName' and table_name = 'yourTableName';



To get the non-null counts of each column in SQL, use this query:

**SELECT COUNT('column name') FROM 'table name'**

**WHERE 'column name' IS NOT NULL;**

In [ ]:
SELECT COUNT('column name') FROM 'table name'

    WHERE 'column name' IS NOT NULL;

In [18]:
# Convert 'ADMITTED_DT' to datetime data type
df['ADMITTED_DT'] = pd.to_datetime(df['ADMITTED_DT'])

df.dtypes


INMATEID                       int64
ADMITTED_DT           datetime64[ns]
DISCHARGED_DT         datetime64[ns]
CUSTODY_LEVEL                 object
BRADH                         object
RACE                          object
GENDER                        object
AGE                            int64
INMATE_STATUS_CODE            object
SEALED                        object
SRG_FLG                       object
TOP_CHARGE                    object
INFRACTION                    object
dtype: object

In [ ]:
UPDATE mytable

SET ADMITTED_DT = CAST(ADMITTED_DT AS DATETIME),


In [22]:
conn = sqlite3.connect('crime_database.db')


In [23]:
df.to_sql('mytable', conn, if_exists='replace', index=False)


6146

In [13]:
query = "SELECT COUNT(*) as Total_Count FROM mytable "
result = pd.read_sql_query(query, conn)
print(result)


   Total_Count
0         6146


In [14]:
query ="""SELECT MIN(Admitted_DT) as Total_Count FROM mytable
"""
result = pd.read_sql_query(query, conn)
print(result)

           Total_Count
0  2012-10-28 14:47:22


In [15]:
import pandas as pd
import sqlite3



# Define the SQL query
query = "SELECT COUNT(*) as Total_Columns FROM pragma_table_info('mytable')"

# Execute the query and store the result in a DataFrame
result = pd.read_sql_query(query, conn)

# Print the result
print(result)


   Total_Columns
0             13


    cid                name       type  notnull dflt_value  pk
0     0            INMATEID    INTEGER        0       None   0
1     1         ADMITTED_DT  TIMESTAMP        0       None   0
2     2       DISCHARGED_DT  TIMESTAMP        0       None   0
3     3       CUSTODY_LEVEL       TEXT        0       None   0
4     4               BRADH       TEXT        0       None   0
5     5                RACE       TEXT        0       None   0
6     6              GENDER       TEXT        0       None   0
7     7                 AGE    INTEGER        0       None   0
8     8  INMATE_STATUS_CODE       TEXT        0       None   0
9     9              SEALED       TEXT        0       None   0
10   10             SRG_FLG       TEXT        0       None   0
11   11          TOP_CHARGE       TEXT        0       None   0
12   12          INFRACTION       TEXT        0       None   0


INMATEID               int64
ADMITTED_DT           object
DISCHARGED_DT         object
CUSTODY_LEVEL         object
BRADH                 object
RACE                  object
GENDER                object
AGE                    int64
INMATE_STATUS_CODE    object
SEALED                object
SRG_FLG               object
TOP_CHARGE            object
INFRACTION            object
dtype: object


In [24]:
q1 = """SELECT count(Race) From df 
        Where Race is not null"""

result = ps.sqldf(q1, locals())
result

,count(Race)
0,6128


In [ ]:
-- High-Level EDA
SELECT COUNT(*) AS total_rows FROM your_table;
SELECT column_name, COUNT(*) AS count FROM your_table GROUP BY column_name;
SELECT column_name, COUNT(*) AS missing_values FROM your_table WHERE column_name IS NULL;
SELECT column_name, COUNT(DISTINCT column_name) AS unique_values FROM your_table;

-- Column-Specific EDA
SELECT MIN(numeric_column) AS min_value, MAX(numeric_column) AS max_value, AVG(numeric_column) AS avg_value FROM your_table;
SELECT category_column, COUNT(*) AS count FROM your_table GROUP BY category_column;
SELECT date_column, COUNT(*) AS count FROM your_table WHERE date_column BETWEEN '2023-01-01' AND '2023-12-31';
SELECT numerical_column_1, numerical_column_2, CORR(numerical_column_1, numerical_column_2) AS correlation FROM your_table;
